<a href="https://colab.research.google.com/github/facundodeza/transfomer-audio-classification/blob/master/audio_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# We load the requiere libraries
import numpy as np
import pandas as pd 
import os
import librosa
import matplotlib.pyplot as plt
import gc

from tqdm import tqdm, tqdm_notebook
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

import zipfile


tqdm.pandas()

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

In [0]:
#We load csv with files name. We will use for reading the data
df = pd.read_csv('../input/freesound-audio-tagging-2019/train_curated.csv')
#df_noisy = pd.read_csv('../input/freesound-audio-tagging-2019/train_noisy.csv')

#train=pd.concat([df, df_noisy], axis=0)

test_df = pd.read_csv('../input/freesound-audio-tagging-2019/sample_submission.csv')


In [0]:
# We get the labels from the test data, we start from 1 because 0 is the file name
# Also, test set only has 80 classes and want to predict according to that.
# Of course, you could create 
# From here we will create Y target

label_columns = test_df.columns[1:]
labels = dict((label, index) for index, label in enumerate(label_columns))








In [0]:
## We can see that several targets are in same space. We need to reshape it into a hot encode format
print(df['labels'][:10])

0                     Bark
1                 Raindrop
2          Finger_snapping
3                      Run
4          Finger_snapping
5               Whispering
6    Acoustic_guitar,Strum
7                   Hi-hat
8                Bass_drum
9           Crowd,Cheering
Name: labels, dtype: object


In [0]:
#We split the labels
y_helper=df['labels'].str.split()

#We say it our labels
mlb=MultiLabelBinarizer(classes=list(labels.keys()))

#We create our hot encoder target
Y=mlb.fit_transform(y_helper)



/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:951: UserWarning: unknown class(es) ['Acoustic_guitar,Strum', 'Applause,Cheering', 'Applause,Cheering,Clapping', 'Applause,Clapping', 'Applause,Crowd', 'Applause,Crowd,Cheering', 'Applause,Crowd,Clapping', 'Applause,Dishes_and_pots_and_pans,Crowd,Cheering,Clapping,Child_speech_and_kid_speaking', 'Applause,Harmonica', 'Applause,Harmonica,Cheering', 'Applause,Screaming,Crowd', 'Bark,Chirp_and_tweet', 'Bark,Cricket', 'Bark,Cricket,Buzz', 'Bark,Walk_and_footsteps', 'Bathtub_(filling_or_washing),Child_speech_and_kid_speaking', 'Bathtub_(filling_or_washing),Gurgling', 'Bathtub_(filling_or_washing),Sink_(filling_or_washing)', 'Bathtub_(filling_or_washing),Sink_(filling_or_washing),Water_tap_and_faucet', 'Bathtub_(filling_or_washing),Water_tap_and_faucet', 'Burping_and_eructation,Chewing_and_mastication', 'Car_passing_by,Accelerating_and_revving_and_vroom', 'Car_passing_by,Race_car_and_auto_racing', 'Car_passing_by,Traffic_n

In [0]:
# unzipping our files


with zipfile.ZipFile('../input/freesound-audio-tagging-2019/train_curated.zip', 'r') as zip_ref:
    zip_ref.extractall('../output/kaggle/working/train')

In [0]:
#import zipfile
#with zipfile.ZipFile('../input/freesound-audio-tagging-2019/train_noisy.zip', 'r') as zip_ref:
 #   zip_ref.extractall('../output/kaggle/working/train')

    #


In [0]:
## We define this parameters for our audio processing

class conf:
    # Preprocessing settings
    sampling_rate = 44100
    duration = 5
    hop_length = 400
    fmin = 20
    fmax = sampling_rate // 2
    n_mels = 128
    n_fft = n_mels * 20
    samples = sampling_rate * duration




In [0]:
train_path = '../output/kaggle/working/train'
#train_noisy_path = '../input/train_noisy/'


In [0]:
## We read the audio data, and trim or padded, if it is needed
## This code was borrowed from https://www.kaggle.com/christofhenkel/keras-baseline-lstm-attention-5-fold

def read_audio(conf, pathname, trim_long_data):
    y, sr = librosa.load(pathname, sr=conf.sampling_rate)
    # trim silence
    if 0 < len(y): # workaround: 0 length causes error
        y, _ = librosa.effects.trim(y) # trim, top_db=default(60)
    # make it unified length to conf.samples
    if len(y) > conf.samples: # long enough
        if trim_long_data:
            y = y[0:0+conf.samples]
    else: # pad blank
        padding = conf.samples - len(y)    # add padding at both ends
        offset = padding // 2
        y = np.pad(y, (offset, conf.samples - len(y) - offset), 'constant')
    return y


## We convert our raw audio into a mel spectogram
def audio_to_melspectrogram(conf, audio):
    spectrogram = librosa.feature.melspectrogram(audio, 
                                                 sr=conf.sampling_rate,
                                                 n_mels=conf.n_mels,
                                                 hop_length=conf.hop_length,
                                                 n_fft=conf.n_fft,
                                                 fmin=conf.fmin,
                                                 fmax=conf.fmax)
    spectrogram = librosa.power_to_db(spectrogram)
    spectrogram = spectrogram.astype(np.float32)
    return spectrogram

#This implement the previous two function for one file
def read_as_melspectrogram(conf, pathname, trim_long_data, debug_display=False):
    x = read_audio(conf, pathname, trim_long_data)
    mels = audio_to_melspectrogram(conf, x)
    if debug_display:
        IPython.display.display(IPython.display.Audio(x, rate=conf.sampling_rate))
        show_melspectrogram(conf, mels)
    return mels

#This generalizes for all the audio files you have
def convert_wav_to_image(df, source):
    X = []
    for i, row in tqdm_notebook(df.iterrows()):
        try:
            x = read_as_melspectrogram(conf, f'{source[0]}/{str(row.fname)}', trim_long_data=True)
        except:
            x = read_as_melspectrogram(conf, f'{source[1]}/{str(row.fname)}', trim_long_data=True)

        X.append(x.transpose())
    return X

In [0]:
#We get all our spectrograms
X = np.array(convert_wav_to_image(df, source=[train_path]))


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:45: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [0]:
def scaled_dot_product_attention(query, key, value, mask):
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask zero out padding tokens.
  if mask is not None:
    logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(logits, axis=-1)

  return tf.matmul(attention_weights, value)

In [0]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    outputs = self.dense(concat_attention)

    return outputs

In [0]:
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [0]:
# This allows to the transformer to know where there is real data and where it is padded
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [0]:
def encoder_layer(units, d_model, num_heads, dropout,name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None,d_model ), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [0]:
def encoder(time_steps,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            projection,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")
  
  if projection=='linear':
    ## We implement a linear projection based on Very Deep Self-Attention Networks for End-to-End Speech Recognition. Retrieved from https://arxiv.org/abs/1904.13377
    projection=tf.keras.layers.Dense( d_model,use_bias=True, activation='linear')(inputs)
  
  else:
    projection=tf.identity(inputs)
   
  projection *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  projection = PositionalEncoding(time_steps, d_model)(projection)

  outputs = tf.keras.layers.Dropout(rate=dropout)(projection)

  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])
 
 
  

 
  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [0]:
def transformer(time_steps,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                output_size,
                projection,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  
  
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(tf.dtypes.cast(
          
    #Like our input has a dimension of length X d_model but the masking is applied to a vector
    # We get the sum for each row and result is a vector. So, if result is 0 it is because in that position was masked      
    tf.math.reduce_sum(
    inputs,
    axis=2,
    keepdims=False,
    name=None
), tf.int32))
  

  enc_outputs = encoder(
      time_steps=time_steps,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      projection=projection,
      name='encoder'
  )(inputs=[inputs, enc_padding_mask])

  #We reshape for feeding our FC in the next step
  outputs=tf.reshape(enc_outputs,(-1,time_steps*d_model))
  
  #We predict our class
  outputs = tf.keras.layers.Dense(units=output_size,use_bias=True,activation='softmax', name="outputs")(outputs)

  return tf.keras.Model(inputs=[inputs], outputs=outputs, name='audio_class')

In [0]:
#We get our train and test set
X_train,X_test, Y_train, Y_test =train_test_split(X,Y, test_size=0.2, random_state=27)



In [0]:
projection=['linear','none']
accuracy=[]
proj_implemented=[]

In [0]:
for i in projection:
  NUM_LAYERS = 2
  D_MODEL = X.shape[2]
  NUM_HEADS = 4
  UNITS = 1024
  DROPOUT = 0.1
  TIME_STEPS= X.shape[1]
  OUTPUT_SIZE=80
  EPOCHS = 10
  EXPERIMENTS=10

  
  for j in range(EXPERIMENTS):
    
    
    model = transformer(time_steps=TIME_STEPS,
      num_layers=NUM_LAYERS,
      units=UNITS,
      d_model=D_MODEL,
      num_heads=NUM_HEADS,
      dropout=DROPOUT,
      output_size=OUTPUT_SIZE,  
      projection=i  )
    
    model.compile(optimizer=tf.keras.optimizers.Adam(0.000001), loss='categorical_crossentropy', metrics=['accuracy'])


    history=model.fit(X_train,Y_train, epochs=EPOCHS, validation_data=(X_test, Y_test))

    accuracy.append(max(history.history['val_accuracy']))
      
    proj_implemented.append(i)
    
    del model
    
    del history
      
   


In [0]:
accuracy=pd.DataFrame(accuracy, columns=['accuracy'])
proj_implemented=pd.DataFrame(proj_implemented, columns=['projection'])
results=pd.concat([accuracy,proj_implemented],axis=1)


In [0]:
results.groupby('projection').mean()


In [0]:
results.groupby('projection').std()